In [22]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import googlemaps
import time

In [24]:
# https://googlemaps.github.io/google-maps-services-python/docs/index.html
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

In [25]:
victoria_bc_canada_coords = "48.407326,-123.329773"

def get_places_nearby(keyword):
    data = []
    page_token = None
    results = {"next_page_token": True}
    page = 1
    while 'next_page_token' in results:
        print(f"Searching: {keyword} | page {page}")
        results = gmaps.places_nearby(location=victoria_bc_canada_coords, radius=6000, keyword=keyword, page_token=page_token)
        
        if 'next_page_token' in results:
            page_token = results['next_page_token']
        
        for result in results['results']:
            row = {
                'place_id': result['place_id'],
                'name': result['name'],
                
                'avg_rating': result['rating'] if 'rating' in result else np.nan,
                
                'lat': result['geometry']['location']['lat'],
                'lng': result['geometry']['location']['lng'],
                
                'user_ratings_total': result['user_ratings_total'] if 'user_ratings_total' in result else np.nan,
            }
            data.append(row)
        time.sleep(2)
        page += 1

    return data

In [26]:
# t = ["1", "2", "3"]
# blob = "1 \n 2 \n 3"

def get_place_details(place_data_df):
    for index, row in place_data_df.iterrows():
        place_details = gmaps.place(place_id=row['place_id'], 
                                   fields=['reviews', 'place_id'])
        
        if "reviews" in place_details['result']:
            review_blob = ""
            for review in place_details['result']['reviews']:
                review_blob += review['text'] + "\n"
            place_data_df.loc[index, 'reviews'] = review_blob
    
    return place_data_df


In [27]:
def get_place_data(searches):
    data = []
    for search in searches:
        data.extend(get_places_nearby(search))

    df = pd.DataFrame(data)
    print("Scraped: ", df.shape)

    df = df.drop_duplicates(subset=['name'], keep='first')
    print("Unique:", df.shape)

    df = get_place_details(df)

    return df

# Scraping Locations

In [28]:
# https://developers.google.com/maps/documentation/places/web-service/supported_types

location_searches = ["things to do", "tourist attractions", "scenic view points", "historical sites", "beaches", "clothing stores", "restaurants", "breakfast", "lunch", "dinner", "rentals", "biking"]

In [29]:
df_locations = get_place_data(location_searches)

Searching: things to do | page 1


Searching: things to do | page 2
Searching: tourist attractions | page 1
Searching: tourist attractions | page 2
Searching: tourist attractions | page 3
Searching: scenic view points | page 1
Searching: historical sites | page 1
Searching: historical sites | page 2
Searching: beaches | page 1
Searching: clothing stores | page 1
Searching: clothing stores | page 2
Searching: clothing stores | page 3
Searching: restaurants | page 1
Searching: restaurants | page 2
Searching: restaurants | page 3
Searching: breakfast | page 1
Searching: breakfast | page 2
Searching: breakfast | page 3
Searching: lunch | page 1
Searching: lunch | page 2
Searching: lunch | page 3
Searching: dinner | page 1
Searching: dinner | page 2
Searching: dinner | page 3
Searching: rentals | page 1
Searching: biking | page 1
Searching: biking | page 2
Scraped:  (504, 6)
Unique: (357, 6)


In [30]:
df_locations.head()

,place_id,name,avg_rating,lat,lng,user_ratings_total,reviews
0,ChIJ1e5UJJF0j1QR6VpPNf8TYiQ,Royal BC Museum,4.6,48.419811,-123.367450,8661,Great museum has very nice and extensive exhib...
1,ChIJmXc0Z6J0j1QR4CJUS73dyZk,Fisherman's Wharf Park,4.5,48.421950,-123.383014,7523,A popular tourist attraction on the Victoria w...
2,ChIJueKTCmtzj1QRxJZTfhNWlsY,Esquimalt Gorge Park & Pavilion,4.7,48.446345,-123.404949,1078,This park is so lovely! City staff do a great ...
3,ChIJ5ZScPpF0j1QRBLCY8tTvWsM,Thunderbird Park,4.3,48.419794,-123.366680,203,Like the totems and of course worth visiting t...
4,ChIJS9YYcJB0j1QRPWHIFtwTr18,Victoria Bug Zoo,4.5,48.423300,-123.366668,1822,My daughter and I had a fantastic time at the ...


In [32]:
df_locations.to_csv("locations.csv", index=False)